# Predicción de Precios de Vuelos de Aerolíneas - EDA y Procesamiento de Datos

Este notebook contiene el análisis exploratorio de datos (EDA) y el procesamiento inicial del dataset de precios de vuelos.

## Resumen del Dataset
- **Total de registros**: 300,153 vuelos
- **Variable objetivo**: Price (precio del boleto)
- **Características**: 11 variables (10 predictoras + 1 objetivo)

## Variables del Dataset
1. **Airline**: Aerolínea (6 categorías)
2. **Flight**: Código de vuelo (categórica)
3. **Source City**: Ciudad de origen (6 ciudades)
4. **Departure Time**: Horario de salida (6 períodos)
5. **Stops**: Número de escalas (3 valores)
6. **Arrival Time**: Horario de llegada (6 períodos)
7. **Destination City**: Ciudad de destino (6 ciudades)
8. **Class**: Clase del asiento (Business/Economy)
9. **Duration**: Duración del vuelo en horas (continua)
10. **Days Left**: Días restantes para el vuelo (derivada)
11. **Price**: Precio del boleto (variable objetivo)

## 1. Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 8)

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## 2. Carga de Datos

In [ ]:
# Cargar el dataset
df = pd.read_csv('../data/01_raw/airlines_flights_data.csv')

print(f"Forma del dataset: {df.shape}")
print(f"Total de registros: {len(df):,}")
print(f"Total de características: {df.shape[1]}")

# Primeras 5 filas
df.head()

## 3. Información General del Dataset

In [ ]:
# Información básica
print("=== INFORMACIÓN DEL DATASET ===")
df.info()

print("\n=== ESTADÍSTICAS DESCRIPTIVAS ===")
df.describe(include='all')

In [ ]:
# Verificar valores nulos
print("=== VALORES NULOS ===")
valores_nulos = df.isnull().sum()
porcentajes_nulos = (valores_nulos / len(df)) * 100

datos_faltantes = pd.DataFrame({
    'Valores Nulos': valores_nulos,
    'Porcentaje (%)': porcentajes_nulos
})

datos_faltantes = datos_faltantes[datos_faltantes['Valores Nulos'] > 0].sort_values('Valores Nulos', ascending=False)

if len(datos_faltantes) > 0:
    print(datos_faltantes)
else:
    print("No hay valores nulos en el dataset")

In [ ]:
# Verificar duplicados
duplicados = df.duplicated().sum()
print(f"=== DUPLICADOS ===")
print(f"Registros duplicados: {duplicados:,}")
print(f"Porcentaje de duplicados: {(duplicados/len(df)*100):.2f}%")

if duplicados > 0:
    print("\nPrimeros 5 registros duplicados:")
    print(df[df.duplicated()].head())
else:
    print("No hay registros duplicados")

## 4. Análisis de Variables Categóricas

In [ ]:
# Identificar variables categóricas y numéricas
columnas_categoricas = df.select_dtypes(include=['object']).columns.tolist()
columnas_numericas = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Remover 'index' si está presente en numéricas
if 'index' in columnas_numericas:
    columnas_numericas.remove('index')

print(f"Variables categóricas ({len(columnas_categoricas)}): {columnas_categoricas}")
print(f"Variables numéricas ({len(columnas_numericas)}): {columnas_numericas}")

In [ ]:
# Análisis de variables categóricas
print("=== ANÁLISIS DE VARIABLES CATEGÓRICAS ===")
for col in columnas_categoricas:
    print(f"\n--- {col.upper()} ---")
    print(f"Valores únicos: {df[col].nunique()}")
    print(f"Valores: {df[col].unique().tolist()}")
    
    conteo_valores = df[col].value_counts()
    porcentajes = (conteo_valores / len(df) * 100).round(2)
    
    resumen = pd.DataFrame({
        'Frecuencia': conteo_valores,
        'Porcentaje (%)': porcentajes
    })
    print(resumen.head(10))

In [ ]:
# Visualización de variables categóricas
fig, axes = plt.subplots(3, 3, figsize=(20, 15))
axes = axes.ravel()

for idx, col in enumerate(columnas_categoricas[:9]):
    if idx < len(columnas_categoricas):
        conteo_valores = df[col].value_counts()
        
        # Gráfico de barras
        axes[idx].bar(range(len(conteo_valores)), conteo_valores.values, 
                     color=sns.color_palette('husl', len(conteo_valores)))
        axes[idx].set_title(f'Distribución de {col}', fontsize=14, fontweight='bold')
        axes[idx].set_xlabel(col)
        axes[idx].set_ylabel('Frecuencia')
        
        # Rotar etiquetas si son muy largas
        if len(str(conteo_valores.index[0])) > 8:
            axes[idx].set_xticks(range(len(conteo_valores)))
            axes[idx].set_xticklabels(conteo_valores.index, rotation=45, ha='right')
        else:
            axes[idx].set_xticks(range(len(conteo_valores)))
            axes[idx].set_xticklabels(conteo_valores.index)
        
        # Añadir valores encima de las barras
        for i, v in enumerate(conteo_valores.values):
            axes[idx].text(i, v, f'{v:,}', ha='center', va='bottom')

# Ocultar ejes vacíos
for idx in range(len(columnas_categoricas), 9):
    axes[idx].set_visible(False)

plt.tight_layout()
plt.show()

## 5. Análisis de Variables Numéricas

In [ ]:
# Estadísticas descriptivas detalladas para variables numéricas
print("=== ANÁLISIS DE VARIABLES NUMÉRICAS ===")
estadisticas_numericas = df[columnas_numericas].describe()
print(estadisticas_numericas)

# Información adicional
print("\n=== INFORMACIÓN ADICIONAL ===")
for col in columnas_numericas:
    print(f"\n--- {col.upper()} ---")
    print(f"Rango: {df[col].min()} - {df[col].max()}")
    print(f"Mediana: {df[col].median()}")
    print(f"Moda: {df[col].mode().iloc[0] if len(df[col].mode()) > 0 else 'No hay moda'}")
    print(f"Asimetría: {df[col].skew():.4f}")
    print(f"Curtosis: {df[col].kurtosis():.4f}")

In [ ]:
# Visualización de variables numéricas
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for idx, col in enumerate(columnas_numericas):
    if idx < 6:  # Máximo 6 gráficos
        # Histograma
        axes[idx].hist(df[col], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
        axes[idx].set_title(f'Distribución de {col}', fontsize=14, fontweight='bold')
        axes[idx].set_xlabel(col)
        axes[idx].set_ylabel('Frecuencia')
        axes[idx].grid(True, alpha=0.3)
        
        # Añadir líneas de media y mediana
        valor_medio = df[col].mean()
        valor_mediana = df[col].median()
        axes[idx].axvline(valor_medio, color='red', linestyle='--', label=f'Media: {valor_medio:.2f}')
        axes[idx].axvline(valor_mediana, color='orange', linestyle='--', label=f'Mediana: {valor_mediana:.2f}')
        axes[idx].legend()

# Ocultar ejes vacíos
for idx in range(len(columnas_numericas), 6):
    axes[idx].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Diagramas de caja para detectar valores atípicos
fig, axes = plt.subplots(1, len(columnas_numericas), figsize=(5*len(columnas_numericas), 6))

if len(columnas_numericas) == 1:
    axes = [axes]

for idx, col in enumerate(columnas_numericas):
    diagrama_caja = axes[idx].boxplot(df[col], patch_artist=True)
    axes[idx].set_title(f'Diagrama de Caja - {col}', fontweight='bold')
    axes[idx].set_ylabel('Valores')
    axes[idx].grid(True, alpha=0.3)
    
    # Colorear las cajas
    diagrama_caja['boxes'][0].set_facecolor('lightblue')
    
    # Calcular y mostrar valores atípicos
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    valores_atipicos = df[(df[col] < limite_inferior) | (df[col] > limite_superior)][col]
    print(f"\nValores atípicos en {col}: {len(valores_atipicos)} ({len(valores_atipicos)/len(df)*100:.2f}%)")

plt.tight_layout()
plt.show()

## 6. Análisis de la Variable Objetivo (Price)

In [ ]:
# Análisis detallado de la variable precio
print("=== ANÁLISIS DE LA VARIABLE OBJETIVO: PRICE ===")
print(f"Media: ${df['price'].mean():,.2f}")
print(f"Mediana: ${df['price'].median():,.2f}")
print(f"Moda: ${df['price'].mode().iloc[0]:,.2f}")
print(f"Desviación estándar: ${df['price'].std():,.2f}")
print(f"Rango: ${df['price'].min():,.2f} - ${df['price'].max():,.2f}")
print(f"Coeficiente de variación: {(df['price'].std()/df['price'].mean()*100):.2f}%")

# Percentiles
percentiles = [5, 10, 25, 50, 75, 90, 95, 99]
print("\nPercentiles:")
for p in percentiles:
    valor = df['price'].quantile(p/100)
    print(f"P{p}: ${valor:,.2f}")

In [ ]:
# Visualización de la distribución de precios
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Histograma
axes[0,0].hist(df['price'], bins=100, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].set_title('Distribución de Precios', fontweight='bold')
axes[0,0].set_xlabel('Precio ($)')
axes[0,0].set_ylabel('Frecuencia')
axes[0,0].axvline(df['price'].mean(), color='red', linestyle='--', 
                 label=f'Media: ${df["price"].mean():,.0f}')
axes[0,0].axvline(df['price'].median(), color='orange', linestyle='--', 
                 label=f'Mediana: ${df["price"].median():,.0f}')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Diagrama de Caja
diagrama_caja = axes[0,1].boxplot(df['price'], patch_artist=True)
axes[0,1].set_title('Diagrama de Caja - Precios', fontweight='bold')
axes[0,1].set_ylabel('Precio ($)')
diagrama_caja['boxes'][0].set_facecolor('lightblue')
axes[0,1].grid(True, alpha=0.3)

# Gráfico Q-Q
stats.probplot(df['price'], dist="norm", plot=axes[1,0])
axes[1,0].set_title('Gráfico Q-Q - Precios vs Normal', fontweight='bold')
axes[1,0].grid(True, alpha=0.3)

# Histograma logarítmico
axes[1,1].hist(np.log(df['price']), bins=100, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1,1].set_title('Distribución Logarítmica de Precios', fontweight='bold')
axes[1,1].set_xlabel('log(Precio)')
axes[1,1].set_ylabel('Frecuencia')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Test de normalidad
print("\n=== PRUEBA DE NORMALIDAD ===")
estadistico_shapiro, p_valor_shapiro = stats.shapiro(df['price'].sample(5000))  # Muestra por limitaciones del test
print(f"Prueba Shapiro-Wilk: estadístico={estadistico_shapiro:.6f}, p-valor={p_valor_shapiro:.6f}")
print(f"Distribución normal: {'No' if p_valor_shapiro < 0.05 else 'Sí'}")

## 7. Análisis de Correlaciones

In [ ]:
# Matriz de correlación para variables numéricas
matriz_correlacion = df[columnas_numericas].corr()

print("=== MATRIZ DE CORRELACIÓN ===")
print(matriz_correlacion.round(3))

# Visualización de la matriz de correlación
plt.figure(figsize=(10, 8))
mascara = np.triu(np.ones_like(matriz_correlacion, dtype=bool))
sns.heatmap(matriz_correlacion, mask=mascara, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8})
plt.title('Matriz de Correlación - Variables Numéricas', fontweight='bold', fontsize=16)
plt.tight_layout()
plt.show()

# Correlaciones con la variable objetivo
if 'price' in matriz_correlacion.columns:
    correlaciones_precio = matriz_correlacion['price'].drop('price').sort_values(key=abs, ascending=False)
    print("\n=== CORRELACIONES CON PRICE ===")
    for variable, correlacion in correlaciones_precio.items():
        print(f"{variable}: {correlacion:.4f}")

## 8. Análisis Bivariado: Precio vs Variables Categóricas

In [ ]:
# Análisis precio por categorías
fig, axes = plt.subplots(3, 3, figsize=(20, 18))
axes = axes.ravel()

for idx, col in enumerate(columnas_categoricas[:9]):
    if idx < len(columnas_categoricas):
        # Diagrama de caja
        df.boxplot(column='price', by=col, ax=axes[idx])
        axes[idx].set_title(f'Precio por {col}', fontweight='bold')
        axes[idx].set_xlabel(col)
        axes[idx].set_ylabel('Precio ($)')
        
        # Rotar etiquetas si es necesario
        etiquetas = df[col].unique()
        if len(str(etiquetas[0])) > 8:
            axes[idx].tick_params(axis='x', rotation=45)
        
        # Remover título automático de pandas
        axes[idx].set_title(f'Precio por {col}', fontweight='bold')

# Ocultar ejes vacíos
for idx in range(len(columnas_categoricas), 9):
    axes[idx].set_visible(False)

plt.suptitle('')  # Remover título general
plt.tight_layout()
plt.show()

In [ ]:
# Estadísticas de precio por categoría
print("=== ESTADÍSTICAS DE PRECIO POR CATEGORÍA ===")
for col in columnas_categoricas:
    print(f"\n--- PRECIO POR {col.upper()} ---")
    estadisticas_precio = df.groupby(col)['price'].agg([
        'count', 'mean', 'median', 'std', 'min', 'max'
    ]).round(2)
    estadisticas_precio.columns = ['Conteo', 'Media', 'Mediana', 'Desv.Std', 'Mínimo', 'Máximo']
    print(estadisticas_precio)
    
    # Prueba ANOVA
    grupos = [df[df[col] == cat]['price'].values for cat in df[col].unique()]
    estadistico_f, valor_p = stats.f_oneway(*grupos)
    print(f"ANOVA F-estadístico: {estadistico_f:.4f}, p-valor: {valor_p:.6f}")
    print(f"Diferencia significativa: {'Sí' if valor_p < 0.05 else 'No'}")

## 9. Detección de Valores Atípicos

In [ ]:
# Detección de valores atípicos usando IQR
print("=== DETECCIÓN DE VALORES ATÍPICOS (MÉTODO IQR) ===")

resumen_atipicos = {}

for col in columnas_numericas:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    atipicos = df[(df[col] < limite_inferior) | (df[col] > limite_superior)]
    conteo_atipicos = len(atipicos)
    porcentaje_atipicos = (conteo_atipicos / len(df)) * 100
    
    resumen_atipicos[col] = {
        'conteo': conteo_atipicos,
        'porcentaje': porcentaje_atipicos,
        'limite_inferior': limite_inferior,
        'limite_superior': limite_superior
    }
    
    print(f"\n--- {col.upper()} ---")
    print(f"Límite inferior: {limite_inferior:.2f}")
    print(f"Límite superior: {limite_superior:.2f}")
    print(f"Valores atípicos: {conteo_atipicos:,} ({porcentaje_atipicos:.2f}%)")
    
    if conteo_atipicos > 0:
        print(f"Valores atípicos extremos:")
        atipicos_extremos = atipicos[col].nlargest(5) if conteo_atipicos > 5 else atipicos[col]
        print(atipicos_extremos.values)

# Resumen de valores atípicos
df_atipicos = pd.DataFrame(resumen_atipicos).T
print("\n=== RESUMEN DE VALORES ATÍPICOS ===")
print(df_atipicos.round(2))

## 10. Análisis de Patrones e Insights

In [ ]:
# Top 10 vuelos más caros y más baratos
print("=== ANÁLISIS DE PRECIOS EXTREMOS ===")
print("\n--- TOP 10 VUELOS MÁS CAROS ---")
mas_caros = df.nlargest(10, 'price')[['airline', 'source_city', 'destination_city', 
                                     'class', 'duration', 'price']]
print(mas_caros)

print("\n--- TOP 10 VUELOS MÁS BARATOS ---")
mas_baratos = df.nsmallest(10, 'price')[['airline', 'source_city', 'destination_city', 
                                        'class', 'duration', 'price']]
print(mas_baratos)

In [ ]:
# Análisis de rutas más populares y costosas
print("=== ANÁLISIS DE RUTAS ===")
df['ruta'] = df['source_city'] + ' → ' + df['destination_city']

# Rutas más populares
print("\n--- TOP 10 RUTAS MÁS POPULARES ---")
rutas_populares = df['ruta'].value_counts().head(10)
print(rutas_populares)

# Precio promedio por ruta
print("\n--- TOP 10 RUTAS MÁS CARAS (PRECIO PROMEDIO) ---")
precios_rutas = df.groupby('ruta')['price'].agg(['count', 'mean', 'median']).round(2)
precios_rutas.columns = ['Vuelos', 'Precio_Promedio', 'Precio_Mediano']
rutas_caras = precios_rutas[precios_rutas['Vuelos'] >= 100].sort_values('Precio_Promedio', ascending=False).head(10)
print(rutas_caras)

In [ ]:
# Análisis temporal - Días restantes vs Precio
print("=== ANÁLISIS TEMPORAL: DÍAS RESTANTES VS PRECIO ===")
dias_precio = df.groupby('days_left')['price'].agg(['count', 'mean', 'median', 'std']).round(2)
dias_precio.columns = ['Vuelos', 'Precio_Promedio', 'Precio_Mediano', 'Desviación']
print(dias_precio.head(20))

# Visualización
plt.figure(figsize=(12, 8))
plt.scatter(df['days_left'], df['price'], alpha=0.5, s=1)
plt.xlabel('Días Restantes')
plt.ylabel('Precio ($)')
plt.title('Relación entre Días Restantes y Precio', fontweight='bold', fontsize=14)
plt.grid(True, alpha=0.3)

# Añadir línea de tendencia
z = np.polyfit(df['days_left'], df['price'], 1)
p = np.poly1d(z)
plt.plot(df['days_left'].sort_values(), p(df['days_left'].sort_values()), 
         "r--", alpha=0.8, linewidth=2)
plt.show()

# Correlación
correlacion = df['days_left'].corr(df['price'])
print(f"\nCorrelación días_restantes vs precio: {correlacion:.4f}")

## 11. Preparación para Preprocesamiento

In [ ]:
# Resumen de hallazgos para preprocesamiento
print("=== RESUMEN DE HALLAZGOS PARA PREPROCESAMIENTO ===")
print("\n1. CALIDAD DE DATOS:")
print(f"   - Sin valores nulos: Sí")
print(f"   - Sin duplicados: Sí" if df.duplicated().sum() == 0 else f"   - Duplicados encontrados: {df.duplicated().sum():,}")

print("\n2. VARIABLES CATEGÓRICAS:")
for col in columnas_categoricas:
    conteo_unico = df[col].nunique()
    print(f"   - {col}: {conteo_unico} categorías únicas")

print("\n3. VARIABLES NUMÉRICAS:")
for col in columnas_numericas:
    porcentaje_atipicos = resumen_atipicos[col]['porcentaje']
    asimetria = df[col].skew()
    print(f"   - {col}: {porcentaje_atipicos:.1f}% valores atípicos, asimetría: {asimetria:.2f}")

print("\n4. VARIABLE OBJETIVO (PRICE):")
asimetria_precio = df['price'].skew()
print(f"   - Distribución: {'Asimétrica positiva' if asimetria_precio > 1 else 'Normal' if abs(asimetria_precio) < 1 else 'Asimétrica negativa'}")
print(f"   - Asimetría: {asimetria_precio:.3f}")
print(f"   - Rango: ${df['price'].min():,.0f} - ${df['price'].max():,.0f}")

print("\n5. RECOMENDACIONES PARA PREPROCESAMIENTO:")
print("   - Aplicar Label Encoding a variables categóricas")
print("   - Considerar normalización/estandarización para variables numéricas")
print(f"   - Evaluar transformación logarítmica para price (asimetría={asimetria_precio:.2f})")
print("   - Analizar tratamiento de valores atípicos según el modelo a usar")
print("   - Crear características adicionales a partir de datetime si es necesario")

## 12. Funciones de Preprocesamiento

In [ ]:
def procesar_datos_aerolineas(datos):
    """
    Función principal de preprocesamiento para el dataset de aerolíneas.
    
    Pasos:
    1. Limpiar y validar datos
    2. Convertir tipos de datos
    3. Manejar valores atípicos si es necesario
    4. Crear características adicionales
    
    Args:
        datos (pd.DataFrame): Dataset original
    
    Returns:
        pd.DataFrame: Dataset procesado
    """
    
    # Crear una copia para no modificar el original
    df_procesado = datos.copy()
    
    print("Iniciando preprocesamiento...")
    
    # 1. Limpiar nombres de columnas (convertir a minúsculas)
    df_procesado.columns = df_procesado.columns.str.lower()
    
    # 2. Remover columna index si existe
    if 'index' in df_procesado.columns:
        df_procesado = df_procesado.drop('index', axis=1)
    
    # 3. Convertir variables categóricas a tipo category (ahorra memoria)
    columnas_categoricas = ['airline', 'flight', 'source_city', 'departure_time', 
                          'stops', 'arrival_time', 'destination_city', 'class']
    
    for col in columnas_categoricas:
        if col in df_procesado.columns:
            df_procesado[col] = df_procesado[col].astype('category')
    
    # 4. Asegurar que variables numéricas sean del tipo correcto
    columnas_numericas = ['duration', 'days_left', 'price']
    for col in columnas_numericas:
        if col in df_procesado.columns:
            df_procesado[col] = pd.to_numeric(df_procesado[col], errors='coerce')
    
    # 5. Validar que no hay valores nulos después del procesamiento
    conteo_nulos = df_procesado.isnull().sum().sum()
    if conteo_nulos > 0:
        print(f"Advertencia: Se encontraron {conteo_nulos} valores nulos después del procesamiento")
    
    # 6. Crear características adicionales
    if 'ruta' not in df_procesado.columns:
        df_procesado['ruta'] = (df_procesado['source_city'].astype(str) + '_a_' + 
                               df_procesado['destination_city'].astype(str))
        df_procesado['ruta'] = df_procesado['ruta'].astype('category')
    
    print(f"Preprocesamiento completado. Forma: {df_procesado.shape}")
    
    return df_procesado

def dividir_datos(datos, tamano_prueba=0.2, tamano_validacion=0.2, semilla_aleatoria=42):
    """
    Divide el dataset en conjuntos de entrenamiento, validación y prueba.
    
    Args:
        datos (pd.DataFrame): Dataset a dividir
        tamano_prueba (float): Proporción para el conjunto de prueba
        tamano_validacion (float): Proporción para el conjunto de validación (del total)
        semilla_aleatoria (int): Semilla para reproducibilidad
    
    Returns:
        tuple: (entrenamiento_df, validacion_df, prueba_df)
    """
    from sklearn.model_selection import train_test_split
    
    # Primera división: separar prueba
    entrenamiento_validacion, prueba = train_test_split(
        datos, test_size=tamano_prueba, random_state=semilla_aleatoria, 
        stratify=datos['class']
    )
    
    # Segunda división: separar entrenamiento y validación
    tamano_val_ajustado = tamano_validacion / (1 - tamano_prueba)  # Ajustar el tamaño de validación
    entrenamiento, validacion = train_test_split(
        entrenamiento_validacion, test_size=tamano_val_ajustado, 
        random_state=semilla_aleatoria, stratify=entrenamiento_validacion['class']
    )
    
    print(f"División completada:")
    print(f"  - Entrenamiento: {len(entrenamiento):,} ({len(entrenamiento)/len(datos)*100:.1f}%)")
    print(f"  - Validación: {len(validacion):,} ({len(validacion)/len(datos)*100:.1f}%)")
    print(f"  - Prueba: {len(prueba):,} ({len(prueba)/len(datos)*100:.1f}%)")
    
    return (entrenamiento.reset_index(drop=True), 
            validacion.reset_index(drop=True), 
            prueba.reset_index(drop=True))

# Ejemplo de uso
print("\n=== EJEMPLO DE PREPROCESAMIENTO ===")
df_procesado = procesar_datos_aerolineas(df)
print(f"\nDataset original: {df.shape}")
print(f"Dataset procesado: {df_procesado.shape}")
print(f"\nColumnas procesadas: {list(df_procesado.columns)}")
print(f"\nTipos de datos:")
print(df_procesado.dtypes)

## 13. Guardar Resultados del EDA

In [ ]:
# Guardar dataset procesado
df_procesado.to_csv('../data/02_intermediate/airlines_procesado_eda.csv', index=False)
print("Dataset procesado guardado en: data/02_intermediate/airlines_procesado_eda.csv")

# Guardar resumen de estadísticas
resumen_estadisticas = {
    'total_registros': len(df),
    'caracteristicas': df.shape[1],
    'variables_categoricas': len(columnas_categoricas),
    'variables_numericas': len(columnas_numericas),
    'valores_nulos': df.isnull().sum().sum(),
    'duplicados': df.duplicated().sum(),
    'rango_precio': f"${df['price'].min():,.0f} - ${df['price'].max():,.0f}",
    'precio_medio': df['price'].mean(),
    'precio_std': df['price'].std(),
    'precio_asimetria': df['price'].skew()
}

import json
with open('../data/08_reporting/resumen_eda.json', 'w', encoding='utf-8') as f:
    json.dump(resumen_estadisticas, f, indent=2, default=str, ensure_ascii=False)

print("Resumen de EDA guardado en: data/08_reporting/resumen_eda.json")
print("\n=== EDA COMPLETADO ===")
print("Este notebook ha analizado completamente el dataset de vuelos.")
print("Los próximos pasos serían:")
print("1. Ingeniería de características detallada")
print("2. Entrenamiento de modelos de ML")
print("3. Evaluación y comparación de modelos")